<a href="https://colab.research.google.com/github/n70l/T5_Tasks/blob/main/RAG_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [49]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [50]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

In [51]:
!pip install langchain langchain_community unstructured sentence_transformers tiktoken chromadb langchain_chroma langchain_groq

In [52]:
import pandas as pd
import os
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq

# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [165]:
df = pd.read_csv("/content/Dataset.csv")
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [166]:
df.sample(10)

,المخالفة,الغرامة
102,طمس رقم هيكل المركبة أو محاولة طمسه.,الغرامة المالية 5000 - 10000 ريال
85,تجاوز حافلات النقل المدرسي عند توقفها للتحميل ...,الغرامة المالية 3000 - 6000 ريال
75,التجاوز في المناطق التي يمنع فيها التجاوز، مثل...,الغرامة المالية 1000 - 2000 ريال
16,وضع حواجز داخل المركبة أو خارجها تعوق رؤية الس...,الغرامة المالية 150 - 300 ريال
7,عدم استخدام إشارة الالتفاف عند التحول لليمين أ...,الغرامة المالية 150 - 300 ريال
50,عدم إعطاء أفضلية المرور لوسائل النقل العام كال...,الغرامة المالية 500 - 900 ريال
56,تظليل زجاج المركبة دون التقيد بالضوابط التي تض...,الغرامة المالية 500 - 900 ريال
58,عدم تغطية الحمولة المنقولة وتثبيتها.,الغرامة المالية 500 - 900 ريال
88,استخدام أجهزة غير مصرح بها في المركبة أو وضع ش...,الغرامة المالية 3000 - 6000 ريال
82,قيادة مركبة دون توافر التجهيزات اللازمة، مثل ا...,الغرامة المالية 1000 - 2000 ريال


In [167]:
df.tail()

,المخالفة,الغرامة
99,القيام بأعمال الطرق قبل التنسيق مع الإدارة الم...,الغرامة المالية 5000 - 10000 ريال
100,سماح أصحاب الحيوانات بعبور حيواناتهم من غير ال...,الغرامة المالية 5000 - 10000 ريال
101,استخدام لوحات غير عائدة للمركبة.,الغرامة المالية 5000 - 10000 ريال
102,طمس رقم هيكل المركبة أو محاولة طمسه.,الغرامة المالية 5000 - 10000 ريال
103,المجازفة بعبور الأودية والشعاب أثناء جريانها.,الغرامة المالية 5000 - 10000 ريال


In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   المخالفة  104 non-null    object
 1   الغرامة   104 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [169]:
directory = 'markdown_files'
os.makedirs(directory, exist_ok=True)

In [170]:
for i in range(0,104):
  title = df['المخالفة'].iloc[i]
  content = df['الغرامة'].iloc[i]
  mark_content = f"# {title}\n\n"
  mark_content += f"{content}\n\n"

  with open(f'{directory}/{i}.md','w',encoding='utf=8') as file:
    file.write(mark_content)

In [171]:
mark_content

'# المجازفة بعبور الأودية والشعاب أثناء جريانها.\n\nالغرامة المالية 5000 - 10000 ريال\n\n'

In [172]:
markdown_texts = []
for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      mark_content = file.read()
      html_content = markdown.markdown(mark_content)
      markdown_texts.append(html_content)

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [173]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.create_documents(markdown_texts)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [174]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(documents, embedding_function, persist_directory="./chroma_db")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [176]:
from langchain.prompts import PromptTemplate
template = """

استخدم المعلومات التالية للإجابة على سؤال المستخدم بأفضل طريقة ممكنة
بناء على السياق و السؤال استخرج الإجابة,

أجب دائما باللغة العربية

السياق:
{context}

السؤال:
{question}

إجابة:
"""
prompt_template = PromptTemplate(template=template, input_variables=["context", "question"])

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [177]:
groq_api_key = "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"
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [178]:
model = LLMChain(llm=llm, prompt=prompt_template, verbose=True)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [179]:
def query_rag(query: str):
    similarity_search_results = db.similarity_search_with_score(query, k=3)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    rag_response = model.invoke({"context": context_text, "question": query})

    return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [181]:
response = query_rag("ماهي الغرامة على الرجوع إلى الخلف في الطريق العام؟")

response

Prompt after formatting:


استخدم المعلومات التالية للإجابة على سؤال المستخدم بأفضل طريقة ممكنة
بناء على السياق و السؤال استخرج الإجابة,

أجب دائما باللغة العربية

السياق:
<h1>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.</h1>
<p>الغرامة المالية 150 - 300 ريال</p>

<h1>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.</h1>
<p>الغرامة المالية 150 - 300 ريال</p>

<h1>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.</h1>
<p>الغرامة المالية 150 - 300 ريال</p>

السؤال:
ماهي الغرامة على الرجوع إلى الخلف في الطريق العام؟

إجابة:


> Finished chain.


{'context': '<h1>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.</h1>\n<p>الغرامة المالية 150 - 300 ريال</p>\n\n<h1>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.</h1>\n<p>الغرامة المالية 150 - 300 ريال</p>\n\n<h1>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.</h1>\n<p>الغرامة المالية 150 - 300 ريال</p>',
 'question': 'ماهي الغرامة على الرجوع إلى الخلف في الطريق العام؟',
 'text': 'الغرامة المالية على الرجوع إلى الخلف في الطريق العام تتراوح بين 150 ريال و 300 ريال.'}